# Generatyviniai tinklai

Pasikartojantys neuroniniai tinklai (RNN) ir jų užtvarų ląstelių variantai, tokie kaip ilgos trumpalaikės atminties ląstelės (LSTM) ir užtvarų pasikartojantys vienetai (GRU), suteikė mechanizmą kalbos modeliavimui, t. y. jie gali išmokti žodžių tvarką ir pateikti prognozes apie kitą žodį sekoje. Tai leidžia mums naudoti RNN **generatyvinėms užduotims**, tokioms kaip įprastas teksto generavimas, mašininis vertimas ir net vaizdų aprašymas.

RNN architektūroje, kurią aptarėme ankstesniame skyriuje, kiekvienas RNN vienetas generavo kitą paslėptą būseną kaip išvestį. Tačiau mes taip pat galime pridėti kitą išvestį prie kiekvieno pasikartojančio vieneto, kuris leistų mums generuoti **seką** (kuri yra tokio pat ilgio kaip pradinė seka). Be to, galime naudoti RNN vienetus, kurie kiekviename žingsnyje nepriima įvesties, o tiesiog naudoja pradinį būsenos vektorių ir tada generuoja išvesties seką.

Šiame užrašų knygelėje mes sutelksime dėmesį į paprastus generatyvinius modelius, kurie padeda mums generuoti tekstą. Paprastumo dėlei sukurkime **simbolių lygmens tinklą**, kuris generuoja tekstą raidė po raidės. Mokymo metu mums reikia paimti tam tikrą teksto korpusą ir padalyti jį į raidžių sekas.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Charakterių žodyno kūrimas

Norint sukurti generatyvinį tinklą simbolių lygiu, tekstą reikia suskaidyti į atskirus simbolius, o ne žodžius. `TextVectorization` sluoksnis, kurį naudojome anksčiau, to padaryti negali, todėl turime dvi galimybes:

* Rankiniu būdu įkelti tekstą ir atlikti tokenizaciją „rankomis“, kaip parodyta [šiame oficialiame Keras pavyzdyje](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Naudoti `Tokenizer` klasę simbolių lygio tokenizacijai.

Mes pasirinkome antrąjį variantą. `Tokenizer` taip pat gali būti naudojamas žodžių tokenizacijai, todėl turėtų būti gana paprasta pereiti nuo simbolių lygio prie žodžių lygio tokenizacijos.

Norint atlikti simbolių lygio tokenizaciją, reikia perduoti parametrą `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Mes taip pat norime naudoti vieną specialų žymeklį, kuris žymėtų **sekos pabaigą**, kurį pavadinsime `<eos>`. Pridėkime jį rankiniu būdu į žodyną:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Generatyvios RNN mokymas kurti pavadinimus

Štai kaip mes mokysime RNN generuoti naujienų pavadinimus. Kiekviename žingsnyje imsime vieną pavadinimą, kurį pateiksime RNN, ir kiekvienam įvesties simboliui prašysime tinklo sugeneruoti kitą išvesties simbolį:

![Paveikslėlis, rodantis RNN pavyzdį, generuojant žodį „HELLO“.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Paskutiniam mūsų sekos simboliui prašysime tinklo sugeneruoti `<eos>` žymeklį.

Pagrindinis skirtumas tarp generatyvios RNN, kurią naudojame čia, yra tas, kad imsime išvestį iš kiekvieno RNN žingsnio, o ne tik iš paskutinės ląstelės. Tai galima pasiekti nurodant `return_sequences` parametrą RNN ląstelei.

Taigi, mokymo metu tinklo įvestis bus tam tikro ilgio užkoduotų simbolių seka, o išvestis bus tokio pat ilgio seka, bet paslinkta vienu elementu ir baigiama `<eos>`. Minipartija sudarys kelias tokias sekas, ir mums reikės naudoti **užpildymą**, kad suderintume visas sekas.

Sukurkime funkcijas, kurios transformuos mums duomenų rinkinį. Kadangi norime užpildyti sekas minipartijos lygiu, pirmiausia sugrupuosime duomenų rinkinį naudodami `.batch()`, o tada naudosime `map`, kad atliktume transformaciją. Taigi, transformacijos funkcija priims visą minipartiją kaip parametrą:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Keletas svarbių dalykų, kuriuos čia atliekame:
* Pirmiausia išgauname tikrąjį tekstą iš string tipo tensoriaus
* `text_to_sequences` konvertuoja tekstų sąrašą į sveikųjų skaičių tensorių sąrašą
* `pad_sequences` užpildo tuos tensorius iki jų maksimalaus ilgio
* Galiausiai atliekame vieno karšto kodavimo (one-hot encoding) procesą visiems simboliams, taip pat atliekame poslinkį ir `<eos>` pridėjimą. Netrukus paaiškinsime, kodėl mums reikia vieno karšto koduotų simbolių

Tačiau ši funkcija yra **Pythonic**, t. y. jos negalima automatiškai paversti Tensorflow skaičiavimo grafu. Jei bandysime naudoti šią funkciją tiesiogiai `Dataset.map` funkcijoje, gausime klaidų. Turime šį Pythonic kvietimą apgaubti naudojant `py_function` apvalkalą:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Pastaba**: Skirtumas tarp Pythonic ir Tensorflow transformacijos funkcijų gali atrodyti pernelyg sudėtingas, ir galbūt kyla klausimas, kodėl mes nenaudojame standartinių Python funkcijų duomenų rinkiniui transformuoti prieš perduodant jį į `fit`. Nors tai tikrai galima padaryti, naudojant `Dataset.map` yra didelis privalumas, nes duomenų transformacijos procesas vykdomas naudojant Tensorflow skaičiavimo grafiką, kuris išnaudoja GPU skaičiavimo galimybes ir sumažina poreikį perduoti duomenis tarp CPU/GPU.

Dabar galime sukurti savo generatoriaus tinklą ir pradėti mokymą. Jis gali būti pagrįstas bet kuria pasikartojančia ląstele, kurią aptarėme ankstesniame skyriuje (paprasta, LSTM arba GRU). Mūsų pavyzdyje naudosime LSTM.

Kadangi tinklas kaip įvestį naudoja simbolius, o žodyno dydis yra gana mažas, mums nereikia įterpimo sluoksnio – vieno karšto kodavimo (one-hot-encoded) įvestis gali tiesiogiai patekti į LSTM ląstelę. Išvesties sluoksnis būtų `Dense` klasifikatorius, kuris konvertuos LSTM išvestį į vieno karšto kodavimo simbolių numerius.

Be to, kadangi dirbame su kintamo ilgio sekų duomenimis, galime naudoti `Masking` sluoksnį, kad sukurtume kaukę, kuri ignoruos užpildytą (padded) eilutės dalį. Tai nėra griežtai būtina, nes mums nėra labai svarbu viskas, kas yra už `<eos>` žymos, tačiau naudosime šį sluoksnį, kad įgytume patirties su šio tipo sluoksniais. `input_shape` bus `(None, vocab_size)`, kur `None` nurodo kintamo ilgio seką, o išvesties forma taip pat yra `(None, vocab_size)`, kaip matote iš `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generuojant rezultatą

Dabar, kai modelis yra apmokytas, norime jį panaudoti rezultatui generuoti. Visų pirma, mums reikia būdo dekoduoti tekstą, kuris yra pateiktas kaip skaičių sekos. Tam galėtume naudoti funkciją `tokenizer.sequences_to_texts`; tačiau ji nėra labai efektyvi, kai naudojama simbolių lygmens tokenizacija. Todėl mes paimsime tokenų žodyną iš tokenizer (vadinamą `word_index`), sukursime atvirkštinį žemėlapį ir parašysime savo dekodavimo funkciją:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Dabar pradėsime generavimą. Pirmiausia turime tam tikrą eilutę `start`, kurią užkoduojame į seką `inp`, o tada kiekviename žingsnyje kviesime savo tinklą, kad nustatytume kitą simbolį.

Tinklo išvestis `out` yra vektorius su `vocab_size` elementų, kurie atspindi kiekvieno ženklo tikimybes. Naudodami `argmax` galime rasti labiausiai tikėtiną ženklo numerį. Tada šį simbolį pridedame prie sugeneruoto ženklų sąrašo ir tęsiame generavimą. Šis procesas, kai sugeneruojamas vienas simbolis, kartojamas `size` kartų, kad būtų sugeneruotas reikiamas simbolių skaičius, o generavimą baigiame anksčiau, jei pasiekiamas `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Pavyzdžių generavimas treniruotės metu

Kadangi neturime jokių naudingų metrikų, tokių kaip *tikslumas*, vienintelis būdas pamatyti, ar mūsų modelis tobulėja, yra **generuojamų eilučių pavyzdžių peržiūra** treniruotės metu. Tam naudosime **atšaukimus** (callbacks), t. y. funkcijas, kurias galime perduoti `fit` funkcijai ir kurios bus periodiškai iškviečiamos treniruotės metu.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Šis pavyzdys jau generuoja gana gerą tekstą, tačiau jį galima dar labiau patobulinti keliais būdais:

* **Daugiau teksto**. Mes naudojome tik antraštes savo užduočiai, tačiau galite eksperimentuoti su pilnu tekstu. Atminkite, kad RNN nėra labai gerai pritaikyti ilgiems sekų apdorojimams, todėl verta jas suskaidyti į trumpesnes sakinių dalis arba visada treniruoti fiksuoto sekos ilgio, pvz., `num_chars` (pavyzdžiui, 256). Galite pabandyti pakeisti aukščiau pateiktą pavyzdį į tokią architektūrą, naudodami [oficialų Keras vadovą](https://keras.io/examples/generative/lstm_character_level_text_generation/) kaip įkvėpimą.

* **Daugiasluoksnis LSTM**. Verta išbandyti 2 ar 3 LSTM ląstelių sluoksnius. Kaip minėjome ankstesniame skyriuje, kiekvienas LSTM sluoksnis iš tekstų išskiria tam tikrus raštus, o simbolių lygio generatoriaus atveju galime tikėtis, kad žemesnis LSTM lygis bus atsakingas už skiemenų išskyrimą, o aukštesni lygiai - už žodžius ir jų derinius. Tai galima paprastai įgyvendinti perduodant sluoksnių skaičiaus parametrą LSTM konstruktoriui.

* Taip pat galite eksperimentuoti su **GRU vienetais** ir pažiūrėti, kurie veikia geriau, bei su **skirtingais paslėptų sluoksnių dydžiais**. Per didelis paslėptas sluoksnis gali sukelti per didelį pritaikymą (pvz., tinklas išmoks tikslų tekstą), o mažesnis dydis gali neduoti gero rezultato.


## Minkštas teksto generavimas ir temperatūra

Ankstesnėje `generate` apibrėžtyje mes visada rinkdavomės simbolį su didžiausia tikimybe kaip kitą simbolį generuojamame tekste. Tai lėmė, kad tekstas dažnai „kartodavosi“ tarp tų pačių simbolių sekų vėl ir vėl, kaip šiame pavyzdyje:
```
today of the second the company and a second the company ...
```

Tačiau, jei pažvelgsime į tikimybių pasiskirstymą kitam simboliui, gali būti, kad skirtumas tarp kelių didžiausių tikimybių nėra didelis, pvz., vienas simbolis gali turėti tikimybę 0.2, kitas - 0.19 ir pan. Pavyzdžiui, ieškant kito simbolio sekai '*play*', kitas simbolis gali būti tiek tarpas, tiek **e** (kaip žodyje *player*).

Tai leidžia daryti išvadą, kad ne visada „teisinga“ pasirinkti simbolį su didesne tikimybe, nes pasirinkus antrą pagal dydį tikimybę vis tiek galime gauti prasmingą tekstą. Protingiau yra **imti mėginius** iš tikimybių pasiskirstymo, kurį pateikia tinklo išvestis.

Šis mėginių ėmimas gali būti atliekamas naudojant `np.multinomial` funkciją, kuri įgyvendina vadinamąjį **multinominį pasiskirstymą**. Funkcija, kuri įgyvendina šį **minkštą** teksto generavimą, apibrėžta žemiau:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Mes pristatėme dar vieną parametrą, vadinamą **temperatūra**, kuris naudojamas nurodyti, kaip stipriai turėtume laikytis didžiausios tikimybės. Jei temperatūra yra 1.0, atliekame sąžiningą multinominį mėginių ėmimą, o kai temperatūra pasiekia begalybę - visos tikimybės tampa lygios, ir mes atsitiktinai pasirenkame kitą simbolį. Žemiau pateiktame pavyzdyje galime pastebėti, kad tekstas tampa beprasmiškas, kai temperatūra per daug padidėja, ir primena „ciklinį“ sunkiai generuojamą tekstą, kai ji artėja prie 0.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
